# Converting mapplotblog routines to pandex

This notebook re-implements the warming stripes blog by Maximilian Nöthe:

https://matplotlib.org/matplotblog/posts/warming-stripes/

into a `pandex` extension that can be loaded directly from GitHub


## pandex implementation

The following will download the `heat_stripes` extension located in a public GitHub repo:


In [ ]:
import pandex as pd
pd.ext.import_extension('github:connectedblue/pdext_collection -> mpl.heat_stripes')

In [ ]:
# cut and paste data from
# https://www.metoffice.gov.uk/hadobs/hadcrut4/data/current/time_series/HadCRUT.4.6.0.0.annual_ns_avg.txt
# and store in a local file (pandas unable to read directly from the url)
df = pd.read_fwf(
    'HadCRUT.4.6.0.0.annual_ns_avg.txt',
    index_col=0,
    usecols=(0, 1),
    names=['year', 'anomaly'],
    header=None,
)

df.ext.heat_stripes('anomaly', clim=0.7, reference='1971:2000')

## Testing alternate parameter values

In [ ]:
df = pd.read_fwf(
    'HadCRUT.4.6.0.0.annual_ns_avg.txt',
    index_col=2,
    usecols=(0, 1, 2),
    names=['year', 'anomaly', 'idx'],
    header=None,
)
df.ext.heat_stripes('anomaly', clim=0.7, first=1905, last=2012, index='year')

In [ ]:

df = pd.DataFrame({'values': [2,2,5,3,3,3,4,4.5,4.75,5,5.1, 8,9,10,10,10]})
df.ext.heat_stripes('values')

In [ ]:
help(df.ext.heat_stripes)

# This is the logic from the blog

The code from the blog is reproduced below


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection
from matplotlib.colors import ListedColormap

In [ ]:
FIRST = 1850
LAST = 2018  # inclusive

# Reference period for the center of the color scale
FIRST_REFERENCE = 1971
LAST_REFERENCE = 2000
LIM = 0.6 # degrees

In [ ]:
anomaly = df.loc[:,'anomaly'].dropna()
reference = anomaly.loc[FIRST_REFERENCE:LAST_REFERENCE].mean()

In [ ]:
len(anomaly)//2

In [ ]:
anomaly.index[-1]

In [ ]:
anomaly.std()

In [ ]:
# the colors in this colormap come from http://colorbrewer2.org
# the 8 more saturated colors from the 9 blues / 9 reds
cmap = ListedColormap([
    '#08306b', '#08519c', '#2171b5', '#4292c6',
    '#6baed6', '#9ecae1', '#c6dbef', '#deebf7',
    '#fee0d2', '#fcbba1', '#fc9272', '#fb6a4a',
    '#ef3b2c', '#cb181d', '#a50f15', '#67000d',
])

In [ ]:
fig = plt.figure(figsize=(10, 1))

ax = fig.add_axes([0, 0, 1, 1])
ax.set_axis_off()

In [ ]:
# create a collection with a rectangle for each year
col = PatchCollection([
    Rectangle((y, 0), 1, 1)
    for y in range(FIRST, LAST + 1)
])

# set data, colormap and color limits
col.set_array(anomaly)
col.set_cmap(cmap)
col.set_clim(reference - LIM, reference + LIM)
ax.add_collection(col)

In [ ]:
ax.set_ylim(0, 1)
ax.set_xlim(FIRST, LAST + 1)

In [ ]:
fig